# Measuring Quantum Volume

## Introduction

**Quantum Volume (QV)** is a single-number metric that can be measured using a concrete
protocol on near-term quantum computers of modest size. The QV method quantifies
the largest random circuit of equal width and depth that the computer successfully implements.
Quantum computing systems with high-fidelity operations, high connectivity, large calibrated gate
sets, and circuit rewriting toolchains are expected to have higher quantum volumes. 


## The Quantum Volume Protocol

A QV protocol (see [1]) consists of the following steps:

(We should first import the relevant qiskit classes for the demonstration).

In [1]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tqdm.notebook import tqdm

#Import Qiskit classes
import qiskit
from qiskit import assemble, transpile
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, thermal_relaxation_error
from qiskit.providers.aer import AerSimulator
from qiskit.utils import QuantumInstance

#Import the qv function
import qiskit.ignis.verification.quantum_volume as qv

from qiskit import IBMQ
if IBMQ.active_account() is None:
    IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-kaist', group='internal', project='default')
backend = provider.get_backend('ibmq_montreal')
# backend = AerSimulator.from_backend(backend)
quantum_instance = QuantumInstance(backend=backend, shots=2**13)

In [2]:
basis_gates = backend.configuration().basis_gates

### Step 1: Generate QV sequences

It is well-known that quantum algorithms can be expressed as polynomial-sized quantum circuits built from two-qubit unitary gates. Therefore, a model circuit consists of $d$ layers of random permutations of the qubit labels, followed by random two-qubit gates (from $SU(4)$). When the circuit width $m$ is odd, one of the qubits is idle in each layer.

More precisely, a **QV circuit** with **depth $d$** and **width $m$**, is a sequence $U = U^{(d)}...U^{(2)}U^{(1)}$ of $d$ layers:


$$ U^{(t)} = U^{(t)}_{\pi_t(m'-1),\pi_t(m)} \otimes ... \otimes U^{(t)}_{\pi_t(1),\pi_t(2)} $$


each labeled by times $t = 1 ... d$ and acting on $m' = 2 \lfloor n/2 \rfloor$ qubits. 
Each layer is specified by choosing a uniformly random permutation $\pi_t \in S_m$ of the $m$ qubit indices
and sampling each $U^{(t)}_{a,b}$, acting on qubits $a$ and $b$, from the Haar measure on $SU(4)$.

In the following example we have 6 qubits Q0,Q1,Q3,Q5,Q7,Q10. We are going to look at subsets up to the full set
(each volume circuit will be depth equal to the number of qubits in the subset)

In [3]:
# qubit_lists: list of list of qubit subsets to generate QV circuits
qubit_lists = [[0, 1, 2, 3, 5, 8], [0, 1, 2, 3, 5, 8, 11], [0, 1, 2, 3, 5, 8, 11, 14], [0, 1, 2, 3, 5, 8, 11, 14, 13]]
# ntrials: Number of random circuits to create for each subset
ntrials = 100

We generate the quantum volume sequences. We start with a small example (so it doesn't take too long to run).

In [4]:
import warnings
warnings.filterwarnings('ignore')
qv_circs, qv_circs_nomeas = qv.qv_circuits(qubit_lists, ntrials)

As an example, we print the circuit corresponding to the first QV sequence. Note that the ideal circuits are run on the first n qubits (where n is the number of qubits in the subset).

In [5]:
# pass the first trial of the nomeas through the transpiler to illustrate the circuit
#qv_circs_nomeas[0] = qiskit.compiler.transpile(qv_circs_nomeas[0], basis_gates=basis_gates)

In [6]:
qv_circs_nomeas[0][0].draw(fold=-1)

┌───────────────────────────┐
q_0: ┤0                          ├
     │                           │
q_1: ┤1                          ├
     │                           │
q_2: ┤2                          ├
     │  quantum_volume_[6,6,408] │
q_3: ┤3                          ├
     │                           │
q_4: ┤4                          ├
     │                           │
q_5: ┤5                          ├
     └───────────────────────────┘

### Step 2: Simulate the ideal QV circuits

The quantum volume method requires that we know the ideal output for each circuit, so we use the statevector simulator in Aer to get the ideal result.

In [7]:
# sv_sim = qiskit.Aer.get_backend('aer_simulator')
sv_sim = QuantumInstance(backend=AerSimulator(), shots=2**13)
ideal_results = []
for trial in tqdm(range(ntrials)):
    # clear_output(wait=True)
    for qc in qv_circs_nomeas[trial]:
        qc.save_statevector()
    #result = qiskit.execute(qv_circs_nomeas[trial], backend=sv_sim).result()
    result = sv_sim.execute(qv_circs_nomeas[trial])
    ideal_results.append(result)
    # print(f'Simulated trial {trial+1}/{ntrials}')

  0%|          | 0/100 [00:00<?, ?it/s]

Next, we load the ideal results into a quantum volume fitter

In [8]:
qv_fitter = qv.QVFitter(qubit_lists=qubit_lists)
qv_fitter.add_statevectors(ideal_results)

### Step 3: Calculate the heavy outputs

To define when a model circuit $U$ has been successfully implemented in practice, we use the *heavy output* generation problem. The ideal output distribution is $p_U(x) = |\langle x|U|0 \rangle|^2$, 
where $x \in \{0,1\}^m$ is an observable bit-string. 

Consider the set of output probabilities given by the range of $p_U(x)$ sorted in ascending order 
$p_0 \leq p_1 \leq \dots \leq p_{2^m-1}$. The median of the set of probabilities is 
$p_{med} = (p_{2^{m-1}} + p_{2^{m-1}-1})/2$, and the *heavy outputs* are


$$ H_U = \{ x \in \{0,1\}^m \text{ such that } p_U(x)>p_{med} \}.$$


The heavy output generation problem is to produce a set of output strings such that more than two-thirds are heavy.

As an illustration, we print the heavy outputs from various depths and their probabilities (for trial 0):

In [9]:
for qubit_list in qubit_lists:
    l = len(qubit_list)
    print ('qv_depth_'+str(l)+'_trial_0:', qv_fitter._heavy_outputs['qv_depth_'+str(l)+'_trial_0'])

qv_depth_6_trial_0: ['000001', '000010', '000101', '000111', '001000', '001010', '001111', '010000', '010011', '010101', '011000', '011001', '011011', '011101', '100000', '100001', '100010', '100110', '101000', '101011', '101101', '101110', '101111', '110010', '110011', '110101', '110111', '111000', '111001', '111100', '111110', '111111']
qv_depth_7_trial_0: ['0000001', '0000010', '0000011', '0000100', '0000101', '0000110', '0000111', '0001001', '0001010', '0001011', '0001100', '0001110', '0010001', '0010011', '0010100', '0010101', '0010110', '0010111', '0011011', '0011100', '0011101', '0011111', '0100100', '0100110', '0100111', '0101100', '0101111', '0110001', '0110011', '0110101', '0110110', '0110111', '0111100', '0111111', '1000001', '1000010', '1000011', '1000100', '1000110', '1000111', '1001001', '1001100', '1001101', '1001111', '1010000', '1010100', '1010101', '1010110', '1011001', '1011010', '1011110', '1100000', '1100011', '1100100', '1100110', '1100111', '1101010', '1110000', 

In [10]:
for qubit_list in qubit_lists:
    l = len(qubit_list)
    print ('qv_depth_'+str(l)+'_trial_0:', qv_fitter._heavy_output_prob_ideal['qv_depth_'+str(l)+'_trial_0'])


qv_depth_6_trial_0: 0.8802063796903119
qv_depth_7_trial_0: 0.8616476384039718
qv_depth_8_trial_0: 0.8412642596447292
qv_depth_9_trial_0: 0.871370066131173


In [11]:
exp_results = []
for trial in tqdm(range(ntrials)):
    #clear_output(wait=True)
    # t_qcs = transpile(qv_circs[trial], basis_gates=basis_gates, optimization_level=3)
    # qobj = assemble(t_qcs)
    # result = backend.run(qobj, max_parallel_experiments=0, shots=2**13).result()
    result = quantum_instance.execute(qv_circs[trial])
    exp_results.append(result)
    #print(f'Completed trial {trial+1}/{ntrials}')

  0%|          | 0/100 [00:00<?, ?it/s]

### Step 5: Calculate the average gate fidelity

The *average gate fidelity* between the $m$-qubit ideal unitaries $U$ and the executed $U'$ is:


$$ F_{avg}(U,U') = \frac{|Tr(U^{\dagger}U')|^2/2^m+1}{2^m+1}$$



The observed distribution for an implementation $U'$ of model circuit $U$ is $q_U(x)$, and the probability of sampling
a heavy output is:


$$ h_U = \sum_{x \in H_U} q_U(x)$$



As an illustration, we print the heavy output counts from various depths (for trial 0):

In [ ]:
qv_fitter.add_data(exp_results)
for qubit_list in qubit_lists:
    l = len(qubit_list)
    print ('qv_depth_'+str(l)+'_trial_0:', qv_fitter._heavy_output_counts['qv_depth_'+str(l)+'_trial_0'])

### Step 6: Calculate the achievable depth

The probability of observing a heavy output by implementing a randomly selected depth $d$ model circuit is:


$$h_d = \int_U h_U dU$$



The *achievable depth* $d(m)$ is the largest $d$ such that we are confident that $h_d > 2/3$. In other words,


$$ h_1,h_2,\dots,h_{d(m)}>2/3 \text{ and } h_{d(m)+1} \leq 2/3$$



We now convert the heavy outputs in the different trials and calculate the mean $h_d$ and the error for plotting the graph.

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.gca()

# Plot the essence by calling plot_rb_data
qv_fitter.plot_qv_data(ax=ax, show_plt=False)

# Add title and label
ax.set_title('Quantum Volume for up to %d Qubits \n and %d Trials'%(len(qubit_lists[-1]), ntrials), fontsize=18)

plt.show()

### Step 7: Calculate the Quantum Volume


The quantum volume treats the width and depth of a model circuit with equal importance and measures the largest square-shaped (i.e., $m = d$) model circuit a quantum computer can implement successfully on average. 

The *quantum volume* $V_Q$ is defined as


$$\log_2 V_Q = \arg\max_{m} \min (m, d(m))$$



We list the statistics for each depth. For each depth we list if the depth was successful or not and with what confidence interval. For a depth to be successful the confidence interval must be > 97.5%.

In [ ]:
qv_success_list = qv_fitter.qv_success()
qv_list = qv_fitter.ydata
QV = 1
for qidx, qubit_list in enumerate(qubit_lists):
    if qv_list[0][qidx]>2/3:
        if qv_success_list[qidx][0]:
            print("Width/depth %d greater than 2/3 (%f) with confidence %f (successful). Quantum volume %d"%
                  (len(qubit_list),qv_list[0][qidx],qv_success_list[qidx][1],qv_fitter.quantum_volume()[qidx]))
            QV = qv_fitter.quantum_volume()[qidx]
        else:
            print("Width/depth %d greater than 2/3 (%f) with confidence %f (unsuccessful)."%
                  (len(qubit_list),qv_list[0][qidx],qv_success_list[qidx][1]))
    else:
        print("Width/depth %d less than 2/3 (unsuccessful)."%len(qubit_list))

In [ ]:
print ("The Quantum Volume is:", QV) 

### References

[1] Andrew W. Cross, Lev S. Bishop, Sarah Sheldon, Paul D. Nation, and Jay M. Gambetta, *Validating quantum computers using randomized model circuits*, Phys. Rev. A **100**, 032328 (2019). https://arxiv.org/pdf/1811.12926

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table